In [1]:
import pandas as pd
#import gspread
import requests 
import io
import matplotlib.pyplot as plt
import scipy
import numpy as np

#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

In [45]:
arr=['AL','CN','EN','SG']
def read(arr):
    csvs=[]
    for str in arr:
        csvs.append(pd.read_csv('./'+str+'/train', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None,  names = ['Word', 'Tag']))
    return csvs
(ALdata,CNdata,ENdata,SGdata)=read(arr)    

In [47]:
# AL Train Data Set
ALdata.head(15)

,Word,Tag
0,小,B-DISTRICT
1,港,I-DISTRICT
2,四,B-POI
3,方,I-POI
4,家,I-POI
5,园,I-POI
6,17,B-HOUSENO
7,幢,I-HOUSENO
8,1363,B-CELLNO
9,室,I-CELLNO


In [48]:
# CN Train Data Set
CNdata.head(15)

,Word,Tag
0,这,O
1,大概,O
2,是,O
3,所有,O
4,版本,O
5,里,O
6,无忌,B-neutral
7,哥哥,I-neutral
8,对,O
9,敏敏,B-neutral


In [49]:
# EN Train Data Set
ENdata.head(15)

,Word,Tag
0,Municipal,B-NP
1,bonds,I-NP
2,are,B-VP
3,generally,B-ADVP
4,a,B-ADJP
5,bit,I-ADJP
6,safer,I-ADJP
7,than,B-PP
8,corporate,B-NP
9,bonds,I-NP


In [50]:
# SG Train Data Set
SGdata.head(15)

,Word,Tag
0,I'm,O
1,about,O
2,to,O
3,see,O
4,Adam,B-positive
5,Levine,I-positive
6,and,O
7,I'm,O
8,probably,O
9,going,O


In [6]:
# replace the words that appear less than k times in the training set with
# special word token #UNK# before training. This leads to a "modified training set"
# if a word does not appear in the "modified training set", we replace that with #UNK# as well
# change k to whatever value you want
def smooth(s):
    return
#Smoothing of EN
#k = 3
EN = pd.DataFrame(ENdata['Word'].value_counts()< 3)
EN.apply(lambda val: val == True)
k1 = EN.index[EN['Word'] == True].tolist()
ENdata.loc[ENdata['Word'].isin(k1), 'Word'] = '#UNK#'
#ENdata.head(20)

In [7]:
#Smoothing of AL
AL = pd.DataFrame(ALdata['Word'].value_counts()<3)
AL.apply(lambda val: val == True)
k2 = AL.index[AL['Word'] == True].tolist()
ALdata.loc[ALdata['Word'].isin(k2), 'Word'] = '#UNK#'
#ALdata.head(20)

In [8]:
#Smoothing of CN
CN = pd.DataFrame(CNdata['Word'].value_counts()<3)
CN.apply(lambda val: val == True)
k3 = CN.index[CN['Word'] == True].tolist()
CNdata.loc[CNdata['Word'].isin(k3), 'Word'] = '#UNK#'
#CNdata.head(20)

In [9]:
#Smoothing of SG
SG = pd.DataFrame(SGdata['Word'].value_counts()<3)
SG.apply(lambda val: val == True)
k4 = SG.index[SG['Word'] == True].tolist()
SGdata.loc[SGdata['Word'].isin(k4), 'Word'] = '#UNK#'
#SGdata.head(20)

Goal: Obtain a dataset with Word, Tag, Emission

Steps
1) Obtain count of tags per word
2) Find the Count of the different tags across the whole dataset
3) For each tag for each word, find out what the total count for each tag
4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
5) Keep only the word/tag with highest emission value

Finally, export pdf

In [10]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#EN

# 1)
ENWordTagcount = pd.DataFrame({'Count' : ENdata.groupby(['Word', 'Tag']).size()}).reset_index()
#ENWordTagcount

In [11]:
# 2)
ENTagcount = pd.DataFrame({'Global Count' :ENdata['Tag'].value_counts()}).reset_index()
ENTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

#ENTagcount

In [12]:
# 3
ENTotalTagCount = pd.merge(ENWordTagcount, ENTagcount, on='Tag')
#ENTotalTagCount

In [13]:
# 4)
ENTotalTagCount['Emission'] = ENTotalTagCount['Count']/ENTotalTagCount['Global Count']
#ENTotalTagCount

In [14]:
# 5)
idx = ENTotalTagCount.groupby(['Word'])['Emission'].transform(max) == ENTotalTagCount['Emission']
ENmodel = ENTotalTagCount[idx]

In [15]:
#open file

#export file
r = open("./EN/dev.in","r",encoding="utf8")
f = open("./EN/dev.p2.out","w",encoding="UTF-8")

ls = []
for line in r.readlines():
    if (line !='\n'and line.strip('\n') in ENmodel['Word']) : #check if word not newline and exist in model
        f.write(line.strip('\n')+ " " + ENmodel['Tag'].loc[ENmodel['Word']== line.strip('\n')].item() + "\n" )
    elif (line !='\n'): #check if word does not exist in model
        f.write(line.strip('\n')+ " " + ENmodel['Tag'].loc[ENmodel['Word']== "#UNK#"].item() + "\n" )
    else: #check if word is space
        f.write(line)
f.close()
r.close()
    
    #ls.append(line.strip('\n'))
#print(r.readline())
print(ls)

[]


In [16]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#AL

# 1)
ALWordTagcount = pd.DataFrame({'Count' : ALdata.groupby(['Word', 'Tag']).size()}).reset_index()
#ALWordTagcount

In [17]:
# 2)
ALTagcount = pd.DataFrame({'Global Count' :ALdata['Tag'].value_counts()}).reset_index()
ALTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

#ALTagcount

In [18]:
# 3
ALTotalTagCount = pd.merge(ALWordTagcount, ALTagcount, on='Tag')
#ALTotalTagCount

In [19]:
# 4)
ALTotalTagCount['Emission'] = ALTotalTagCount['Count']/ALTotalTagCount['Global Count']
#ALTotalTagCount

In [20]:
# 5)
idx = ALTotalTagCount.groupby(['Word'])['Emission'].transform(max) == ALTotalTagCount['Emission']
ALmodel = ALTotalTagCount[idx]
#ALmodel

In [21]:
ALmodel['Tag'].loc[ALmodel['Word']=='400'].item()

'B-ASSIST'

In [22]:
#open file
r = open("./AL/dev.in","r",encoding="UTF-8")
f = open("./AL/dev.p2.out","w",encoding="UTF-8")

#export file

for line in r.readlines():
    if (line !='\n'and line.strip('\n') in ALmodel['Word']) : 
        f.write(line.strip('\n')+ " " + ALmodel['Tag'].loc[ALmodel['Word']== line.strip('\n')].item() + "\n" )
    elif (line !='\n'):
        f.write(line.strip('\n')+ " " + ALmodel['Tag'].loc[ALmodel['Word']== "#UNK#"].item() + "\n" )
    else:
        f.write(line)
f.close()
r.close()
    
    #ls.append(line.strip('\n'))
#print(r.readline())
print(ls)

[]


In [23]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#CN

# 1)
CNWordTagcount = pd.DataFrame({'Count' : CNdata.groupby(['Word', 'Tag']).size()}).reset_index()
#CNWordTagcount

In [24]:
# 2)
CNTagcount = pd.DataFrame({'Global Count' :CNdata['Tag'].value_counts()}).reset_index()
CNTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

#CNTagcount

In [25]:
# 3
CNTotalTagCount = pd.merge(CNWordTagcount, CNTagcount, on='Tag')
#CNTotalTagCount

In [26]:
# 4)
CNTotalTagCount['Emission'] = CNTotalTagCount['Count']/CNTotalTagCount['Global Count']
#CNTotalTagCount

In [27]:
# 5)
idx = CNTotalTagCount.groupby(['Word'])['Emission'].transform(max) == CNTotalTagCount['Emission']
CNmodel = CNTotalTagCount[idx]
#CNmodel

In [28]:
#open file
#export file 

r = open("./CN/dev.in","r",encoding="utf8")
f = open("./CN/dev.p2.out","w",encoding="UTF-8")

ls = []
for line in r.readlines():
    if (line !='\n'and line.strip('\n') in CNmodel['Word']) : 
        f.write(line.strip('\n')+ " " + CNmodel['Tag'].loc[CNmodel['Word']== line.strip('\n')].item() + "\n" )
    elif (line !='\n'):
        f.write(line.strip('\n')+ " " + CNmodel['Tag'].loc[CNmodel['Word']== "#UNK#"].item() + "\n" )
    else:
        f.write(line)
f.close()
r.close()
    
    #ls.append(line.strip('\n'))
#print(r.readline())
print(ls)

[]


In [29]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#SG

# 1)
SGWordTagcount = pd.DataFrame({'Count' : SGdata.groupby(['Word', 'Tag']).size()}).reset_index()
#SGWordTagcount

In [30]:
# 2)
SGTagcount = pd.DataFrame({'Global Count' :SGdata['Tag'].value_counts()}).reset_index()
SGTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

#SGTagcount

In [31]:
# 3
SGTotalTagCount = pd.merge(SGWordTagcount, SGTagcount, on='Tag')
#SGTotalTagCount

In [32]:
# 4)
SGTotalTagCount['Emission'] = SGTotalTagCount['Count']/SGTotalTagCount['Global Count']
#SGTotalTagCount

In [33]:
# 5)
idx = SGTotalTagCount.groupby(['Word'])['Emission'].transform(max) == SGTotalTagCount['Emission']
SGmodel = SGTotalTagCount[idx]
#SGmodel

In [34]:
#open file
#export file
r = open("./SG/dev.in","r",encoding="utf8")
f = open("./SG/dev.p2.out","w",encoding="UTF-8")

ls = []
for line in r.readlines():
    if (line !='\n'and line.strip('\n') in SGmodel['Word']) : 
        f.write(line.strip('\n')+ " " + SGmodel['Tag'].loc[SGmodel['Word']== line.strip('\n')].item() + "\n" )
    elif (line !='\n'):
        f.write(line.strip('\n')+ " " + SGmodel['Tag'].loc[SGmodel['Word']== "#UNK#"].item() + "\n" )
    else:
        f.write(line)
f.close()
r.close()
    
    #ls.append(line.strip('\n'))
#print(r.readline())
print(ls)

[]


Evaluation Scripts

In [35]:
!py  ./EvalScript/evalResult.py ./EN/dev.out ./EN/dev.p2.out


#Entity in gold data: 13179
#Entity in prediction: 26131

#Correct Entity : 7310
Entity  precision: 0.2797
Entity  recall: 0.5547
Entity  F: 0.3719

#Correct Sentiment : 1
Sentiment  precision: 0.0000
Sentiment  recall: 0.0001
Sentiment  F: 0.0001


In [36]:
!py   ./EvalScript/evalResult.py ./AL/dev.out ./AL/dev.p2.out


#Entity in gold data: 8408
#Entity in prediction: 24663

#Correct Entity : 926
Entity  precision: 0.0375
Entity  recall: 0.1101
Entity  F: 0.0560

#Correct Sentiment : 282
Sentiment  precision: 0.0114
Sentiment  recall: 0.0335
Sentiment  F: 0.0171


In [37]:
!py  ./EvalScript/evalResult.py ./CN/dev.out ./CN/dev.p2.out


#Entity in gold data: 1478
#Entity in prediction: 23531

#Correct Entity : 700
Entity  precision: 0.0297
Entity  recall: 0.4736
Entity  F: 0.0560

#Correct Sentiment : 63
Sentiment  precision: 0.0027
Sentiment  recall: 0.0426
Sentiment  F: 0.0050


In [38]:
!py ./EvalScript/evalResult.py ./SG/dev.out ./SG/dev.p2.out


#Entity in gold data: 4537
#Entity in prediction: 47656

#Correct Entity : 2483
Entity  precision: 0.0521
Entity  recall: 0.5473
Entity  F: 0.0951

#Correct Sentiment : 572
Sentiment  precision: 0.0120
Sentiment  recall: 0.1261
Sentiment  F: 0.0219
